# 노동법


In [1]:
#-*-coding:utf-8-*-

# 연습문제 8-3: 유튜브 댓글 수집하는 크롤러

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
import openpyxl

from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_argument("--headless")


#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("연습문제 : 유투브 영상의 댓글 수집하기")
print("=" *80)


query_txt=input("1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): ")

cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:'))
reple_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇건입니까?: '))

f_dir = input("4.크롤링 결과를 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

print("\n")
print("요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^")



# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )


#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
#path = "C:\\Temp\\chromedriver_240\\chromedriver.exe"
driver = webdriver.Chrome("C:\\Temp\\chromedriver_240\\chromedriver.exe",options=chrome_options)

driver.get('https://www.youtube.com')

time.sleep(5)
element = driver.find_element_by_name("search_query")
element.send_keys(query_txt)
element.submit()
time.sleep(2)

#검색 결과가 첫 화면에 20개가 나옵니다.
# 만약 사용자가 입력한 동영상 개수가 20개가 넘어갈 경우 스크롤다운해서 갯수를
# 먼저 만든 후 URL 을 추출하여 URL 을 완성해야 합니다

def scroll_down(driver):
    #driver.execute_script("window.scrollTo(500,document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0,3000);") # 한페이지 20개씩 출력값
    time.sleep(5)

if reple_cnt < 20 :
        page_cnt = 1 
else :
        page_cnt = math.ceil(cnt/20)
        
if cnt > 20 :
           
        i = 1
        while ( i <=  page_cnt) :
            print("화면을 %s 회 아래로 스크롤 합니다" %i)
            scroll_down(driver)
            time.sleep(1)
            i += 1
      

# 검색된 유튜브 영상의 URL 을 추출합니다

time.sleep(2)

html = driver.page_source
soup1 = BeautifulSoup(html, 'html.parser')

count = 0
item=[]

print("\n")

for i in soup1.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer'):
          
          item.append(i['href'])
          count += 1

          if count == cnt :
              break

print("검색된 영상 중 %s 건 동영상의 댓글을 추출하겠습니다" %cnt)
print("\n")

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

# 수집된 전체 URL 중에서 사용자가 입력한 수만큼의 동영상의 댓글을 수집합니다.
# 먼저 전체 URL 을 완성 -> 동영상 페이지 열기 -> 댓글 수집 -> 다음 동영상 순으로 진행합니다
full_url=[]
url_cnt = 0
for x in range(0,len(item)) :
          url_cnt += 1
          url = 'https://www.youtube.com/' +item[x]
          full_url.append(url)
          if url_cnt == cnt :
            break


play_cnt = 1

url2=[]
reple2=[]
reple3=[]
reple4=[]
writer2=[]
wdate2=[]

wb=openpyxl.Workbook( )
wb.save(fx_name)


for y in range(0,len(full_url)) :
      driver.get(full_url[y])
      time.sleep(10)
      print("\n")
      print("\n")
      print("%s 번째 동영상의 정보를 수집합니다." %play_cnt)

      i = 1
      while ( i <=  page_cnt) :
      #print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(10)
        i += 1
      
      html = driver.page_source
      soup2 = BeautifulSoup(html, 'html.parser')
        
      t_count_0 = soup2.select('#count')
      t_count_1 = t_count_0[1].get_text()
      t_count_2 = t_count_1.replace(",","")
      t_count_3 = re.search("\d+",t_count_2)
      t_count_4 = int(t_count_3.group())

      t_title_1 = soup2.select('#info-contents')
      t_title_2 = t_title_1[0].find('h1').get_text()
      t_title_3 = t_title_2.translate(bmp_map).replace("\n","")
        
        
      try:
        t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()  #.get_text( ).replace("\n","")
      
      except AttributeError:
        continue
      
      else:
        t_view_2 = t_view_1.replace("," ,"")
        t_view_3 = re.search("\d+",t_view_2)
        print(t_view_1)
        t_view_4 = int(t_view_3.group())
        

#       t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()#.get_text( ).replace("\n","")
#       t_view_2 = t_view_1.replace("," ,"")
#       t_view_3 = re.search("\d+",t_view_2)
#       print(t_view_1)
#       t_view_4 = int(t_view_3.group())

      print("=" *80)
      print("%s 번째 동영상의 조회수는 %s 회 이고 수집할 댓글은 총 %s개 입니다" %(play_cnt,t_count_4,t_view_4))  
      print("%s 번째 동영상의 제목은 %s 입니다" %(play_cnt,t_title_3))
      print("=" *80)
      print("\n")
      print("%s 번째 동영상에서 댓글 수집을 시작합니다" %play_cnt)
      print("댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^")
      print("\n")

 #Step 5. 화면을 스크롤해서 아래로 이동합니다

      #page_cnt = math.ceil(t_view_4 / 19)

      i = 1
      while ( i <=  page_cnt+1) :
        print("%s 번째 페이지의 댓글을 수집하는 중입니다~" %i)
        scroll_down(driver)
        time.sleep(0.5)
        i += 1

      #Step 6. 내용을 수집합니다

      time.sleep(2)

      html = driver.page_source
      soup3 = BeautifulSoup(html, 'html.parser')
        
      count = 0
      d2 = 0

      reple_result = soup3.select('#comments > #sections > #contents')

      for a in range(0,reple_cnt+1) :
              count += 1
              
              f = open(ff_name, 'a',encoding='UTF-8')
              f.write("\n")
              f.write("------------------------------------------------------------------"+"\n")

             # 댓글 작성자
              try : 
                writer = reple_result[0].select("#header-author > #author-text")[a].get_text().replace("\n","").strip()
              except IndexError :
                break
              else :
                print("\n")
                print("%s 번째 영상의 %s 번째  댓글 " %(play_cnt,count))
                print("-" *70)

                # 유튜브 URL 주소
                print("1.URL 주소:",full_url[y])
                f.write("1.유투브 URL주소: " + full_url[y] + "\n")
                url2.append(full_url[y])
                
                print("2.댓글 작성자명:", writer)
                f.write("2.댓글 작성자명: " + writer + "\n")
                writer2.append(writer)
              
                # 댓글 작성일자
                wdate = reple_result[0].select("yt-formatted-string.published-time-text > a.yt-simple-endpoint")[a].get_text().replace("\n","")
            
                print("3.댓글 작성일자:",wdate)
                f.write("3.댓글 작성일자: " + wdate + "\n")
                wdate2.append(wdate)
              
                # 댓글 내용
                reple1 = reple_result[0].select('#expander > #content > #content-text')[a].get_text().replace("\n","")
            
                reple2=reple1.translate(bmp_map).replace("\n","")
                print("4.댓글 내용:",reple2)
                f.write("4.댓글 내용: " + reple2 + "\n")
                reple3.append(reple2)
                
              
              f.close( )
              
              if count == reple_cnt :
                  break
              
      time.sleep(2)       
      play_cnt += 1                

 #step 9. 엑셀 형태로 저장하기
    
              
youtube_reple = pd.DataFrame()
youtube_reple['URL 주소']=url2
youtube_reple['댓글작성자명']=pd.Series(writer2)
youtube_reple['댓글작성일자']=pd.Series(wdate2)
youtube_reple['댓글 내용']=pd.Series(reple3)


# csv 형태로 저장하기
youtube_reple.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
youtube_reple.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )


print("\n") 
print("=" *100)
print("1.요청된 총 %s 건 동영상 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

time.sleep(3)

driver.close( )

 

연습문제 : 유투브 영상의 댓글 수집하기
1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): 노동법
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:200
3.각 동영상에서 추출할 댓글은 몇건입니까?: 159
4.크롤링 결과를 저장할 폴더명만 쓰세요(예:c:\temp\):C:\\temp\\노동법\\


요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^
화면을 1 회 아래로 스크롤 합니다
화면을 2 회 아래로 스크롤 합니다
화면을 3 회 아래로 스크롤 합니다
화면을 4 회 아래로 스크롤 합니다
화면을 5 회 아래로 스크롤 합니다
화면을 6 회 아래로 스크롤 합니다
화면을 7 회 아래로 스크롤 합니다
화면을 8 회 아래로 스크롤 합니다
화면을 9 회 아래로 스크롤 합니다
화면을 10 회 아래로 스크롤 합니다


검색된 영상 중 200 건 동영상의 댓글을 추출하겠습니다






1 번째 동영상의 정보를 수집합니다.
댓글 537개
1 번째 동영상의 조회수는 280749 회 이고 수집할 댓글은 총 537개 입니다
1 번째 동영상의 제목은 [노동법 이슈] 2020년 바뀌는 중요한 노동법 변경사항, 7개만 알고 갑시다!! 입니다


1 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
9 번째 페이지의 댓글을 수집하는 중입니다~
10 번째 페이지의 댓글을 수집하는 중입니다~
11 번째 페이지의 댓글을 수집하는 중입니다~


1 번째 영상의 1 번째  댓글 
-------------------------------------



1 번째 영상의 31 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 얘들아승전보울려라
3.댓글 작성일자: 9개월 전
4.댓글 내용: 고용노동직 공무원 시험 준비하는 학생입니다! 깔끔한 정리영상 감사드립니다


1 번째 영상의 32 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 니똥
3.댓글 작성일자: 9개월 전
4.댓글 내용: 12시간 일하는 식당도 많어 임금은 헬인곳도 많고 ㅋ명절? 이딴건 없는곳도 있던데;;


1 번째 영상의 33 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: Eun Ji
3.댓글 작성일자: 8개월 전
4.댓글 내용: 캬 상상만으로도 넘 좋다 ㅋㅋ


1 번째 영상의 34 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 다그닥다그닥
3.댓글 작성일자: 8개월 전
4.댓글 내용: 기업이 노동법을 위반 했을때 회사가 위태로워질 정도로 패널티를 가해야 회사 내부에서 먼저 노동자의 권리를 누릴 수 있게 장려하는 문화가 자리 잡아 체감 하는거지법으로만 만든다고 체감되는게 아닙니다.


1 번째 영상의 35 번째

4.댓글 내용: 좋은정보 굳굳!!


1 번째 영상의 64 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 둠피스트
3.댓글 작성일자: 9개월 전(수정됨)
4.댓글 내용: 돈은 벌어야하는데 강제 시간제한으로 존나빈둥대고 살게됨 이중취업때문에 애매하게 일자리 창출이 아니라 감소  유도임ㅋㅋ


1 번째 영상의 65 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 강준호
3.댓글 작성일자: 9개월 전
4.댓글 내용: 공항 지상조업 근무자들은 기본시간 + 연장 30시간 이상인데 법에 안걸리나요?


1 번째 영상의 66 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 김백건
3.댓글 작성일자: 9개월 전
4.댓글 내용: 노가다는 치외법권이네 노가다하는사람들은 인간도 아니나?


1 번째 영상의 67 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 권영화
3.댓글 작성일자: 9개월 전(수정됨)
4.댓글 내용: 좋은정보  감사합니다.


1 번째 영상의 68 번째  댓글 
--------------------------------------

3.댓글 작성일자: 6개월 전
4.댓글 내용: 최저임금을 받지 않는 근로자들은 식대 중 10,234원을 총급여액에 추가해서 과세로 본다는 건가요???


1 번째 영상의 99 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 히사시부리
3.댓글 작성일자: 9개월 전
4.댓글 내용: 법이있으면 뭐하나법위에 있는 인간들이 더많은데


1 번째 영상의 100 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 황태수
3.댓글 작성일자: 9개월 전
4.댓글 내용: 8시30분~6시30분퇴근 토요일5시30분퇴근 점심1시간 일요일 휴무공휴일 휴무없고요 얼마정도인가요


1 번째 영상의 101 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: ᄏᄏ이영하
3.댓글 작성일자: 9개월 전
4.댓글 내용: 지엘 리베라움(아파트)에서 계약서쓰고 일하는 사람인데 제가 있는 아파트 직원들만은 5명 이상인데한국에 있는 지엘리베라움 전체 직원으로 인원수를 세야하나요 아니면 제가 일하는 곳에서만 인원수를 세야하나요 ?


1 번째 영상의 102 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBi



1 번째 영상의 132 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 리메디오스
3.댓글 작성일자: 8개월 전
4.댓글 내용: 말씀하신 최저입금 188만원 세전 금액인거죠?


1 번째 영상의 133 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 박채희
3.댓글 작성일자: 6개월 전
4.댓글 내용: 파트타임도 식대 10만원 줘야 되나요??


1 번째 영상의 134 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: ppoppi13 7
3.댓글 작성일자: 6개월 전
4.댓글 내용: 처저임금 188원은 세전인가요? 세후인가요?


1 번째 영상의 135 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=XJcxHyrwBiQ
2.댓글 작성자명: 서형복
3.댓글 작성일자: 9개월 전
4.댓글 내용: 교통이 불편해서  교통비를 월급과 따로 지급하는데요이것도 최저임금에 포함 해야 되는건가요?교통비를 빼면 최저임금에 미달하거든요답변 꼭 부탁합니다


1 번째 영상의 136 번째  댓글 
-----------------------------------------------------

...

2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
9 번째 페이지의 댓글을 수집하는 중입니다~
10 번째 페이지의 댓글을 수집하는 중입니다~
11 번째 페이지의 댓글을 수집하는 중입니다~




18 번째 동영상의 정보를 수집합니다.
댓글 0개
18 번째 동영상의 조회수는 16 회 이고 수집할 댓글은 총 0개 입니다
18 번째 동영상의 제목은 [HR Open Class] 채용 단계에서의 노동법 이슈Ⅰ 입니다


18 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
9 번째 페이지의 댓글을 수집하는 중입니다~
10 번째 페이지의 댓글을 수집하는 중입니다~
11 번째 페이지의 댓글을 수집하는 중입니다~




19 번째 동영상의 정보를 수집합니다.




19 번째 동영상의 정보를 수집합니다.
댓글 2개
19 번째 동영상의 조회수는 850 회 이고 수집할 댓글은 총 2개 입니다
19 번째 동영상의 제목은 2019년 공공운수노동자가 노동법 개악을 저지합시다 입니다


19 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 

4.댓글 내용: 유익한 방송 감사합니다. 이번 영상에는 신대표님은 어디 가셨나요?


25 번째 영상의 4 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=2Rzk2RZpmak
2.댓글 작성자명: バイマイアベノミクス
3.댓글 작성일자: 2개월 전
4.댓글 내용: 직원 권고사직 해서 자영업자가 일자리 지원 안정금 지원이 없어질경우, 언제부터 다시 지원금 신청이 가능해지는지 여쭤봐도 괜찮을까요?


25 번째 영상의 5 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=2Rzk2RZpmak
2.댓글 작성자명: 대빵이
3.댓글 작성일자: 2개월 전
4.댓글 내용: 유익합니다!!


25 번째 영상의 6 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=2Rzk2RZpmak
2.댓글 작성자명: 큰곰
3.댓글 작성일자: 2개월 전
4.댓글 내용: 안녕하세요 질문이 있습니다.7월 27일로 계약직 근로계약만료로 해고통보를 받은 상황에서 한 달만 더 연장해달라는 부탁을 거절하면 자발적퇴사가 되어 실업급여를 받지 못하는건가요?


25 번째 영상의 7 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=2Rzk2RZpmak
2.댓글 작성자명: 엉뚱이TV
3.댓글 작성일자: 1개월 전
4.댓글 내용: 출퇴근 사고시 무

3.댓글 작성일자: 10개월 전
4.댓글 내용: 노동자로서 잘보고 배워갑니다.https://www.facebook.com/100027028191961/posts/425479721696297/


27 번째 영상의 11 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=5aSTbU2El7o
2.댓글 작성자명: 김어선
3.댓글 작성일자: 10개월 전(수정됨)
4.댓글 내용: 감상평:   노른자 노동법 해고편   서너번 반복해서 봤습니다.상당히 유용하고 꿀팁정보에  지루하지도 않고 재미와 교육적이라 엄지척 쵝오였음.노른자 노동법 해고편을 널리 공유하고 알려주겠습니다.http://blog.naver.com/ahffk76/221719290381


27 번째 영상의 12 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=5aSTbU2El7o
2.댓글 작성자명: 이기동
3.댓글 작성일자: 10개월 전(수정됨)
4.댓글 내용: 해고예고수당을 간추려서 보니 알기쉽네요http://blog.naver.com/whsxpfl70/221724099176


27 번째 영상의 13 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=5aSTbU2El7o
2.댓글 작성자명: 김블리
3.댓글 작성일자: 10개월 전
저희신랑도 갑작스럽게 부당해고? 당한것 같아 속상해지네요...이미 지난일이지만 이제 잘알았으니 앞으로는 잘대처해야겠습니다 ^^


27 번째 영상의 14 번째  댓글 
--------

3.댓글 작성일자: 10개월 전
4.댓글 내용: 부당하게 해고당하는 일이 없었으면 좋겠습니다http://blog.naver.com/hjy19950909/221722290481


27 번째 영상의 38 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=5aSTbU2El7o
2.댓글 작성자명: 김재민
3.댓글 작성일자: 10개월 전
4.댓글 내용: 알쏭달쏭한 #해고 궁금증누구나가 겪는 일일수도 있기에 알아두면 정말 좋은 내용이네요. https://twitter.com/jaemin1103/status/1199330633341792256?s=20


27 번째 영상의 39 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=5aSTbU2El7o
2.댓글 작성자명: 김상협
3.댓글 작성일자: 10개월 전
근로관계 종료시 불이익을 당하는 일 없이  자신의 정당한 권리를 찾는데 많은 도움이 되는 영상이었어요  https://www.facebook.com/100007042123987s/posts/2491514044426642


27 번째 영상의 40 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=5aSTbU2El7o
2.댓글 작성자명: 몽실멍실
3.댓글 작성일자: 10개월 전
4.댓글 내용: 과거에 비해 부당해고가 쉬어지지 않아서 정말 좋은 노동법인것 같습니다~!!앞으로도 많은 노동자들에게 힘이 되어주세요^^!!응원합니다https://m.facebook.com/story.p

TimeoutException: Message: timeout
  (Session info: headless chrome=86.0.4240.75)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [2]:
              
youtube_reple = pd.DataFrame()
youtube_reple['URL 주소']=url2
youtube_reple['댓글작성자명']=pd.Series(writer2)
youtube_reple['댓글작성일자']=pd.Series(wdate2)
youtube_reple['댓글 내용']=pd.Series(reple3)


# csv 형태로 저장하기
youtube_reple.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
youtube_reple.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )


print("\n") 
print("=" *100)
print("1.요청된 총 %s 건 동영상 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

time.sleep(3)



1.요청된 총 200 건 동영상 리뷰 중에서 실제 크롤링 된 리뷰수는 41 건입니다
2.총 소요시간은 9058.4 초 입니다 
3.파일 저장 완료: txt 파일명 : C:\\temp\\노동법\\2020-10-17-18-05-46-노동법\2020-10-17-18-05-46-노동법.txt 
4.파일 저장 완료: csv 파일명 : C:\\temp\\노동법\\2020-10-17-18-05-46-노동법\2020-10-17-18-05-46-노동법.csv 
5.파일 저장 완료: xls 파일명 : C:\\temp\\노동법\\2020-10-17-18-05-46-노동법\2020-10-17-18-05-46-노동법.xls 
